In [1]:
import sys
import os

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
from forcateri.model.dartsmodels.dartstcnmodel import DartsTCNModel
from forcateri.baltbestapi.baltbestaggregatedapidata import BaltBestAggregatedAPIData
import pandas as pd
from forcateri.data.dataprovider import DataProvider, SeriesRole
from darts.models import TCNModel
from darts.utils.likelihood_models import QuantileRegression
from forcateri.data.timeseries import TimeSeries

In [3]:
%load_ext autoreload
%autoreload 2 

In [27]:
baltbest = BaltBestAggregatedAPIData(local_copy="/home/dior00002/dfki/forcateri/_data/")
# baltbest = BaltBestAggregatedAPIData(
#     name='test', 
#     url="baltbest_url", 
#     local_copy="/home/dior00002/dfki/forcateri/_data/showcase_data.csv",
#     target = 'delta',
#     group_col = 'room_id',
#     time_col = 'rounded_ts',
#     known = 'outside_temp',
#     observed = ['max_temperature_1', 'max_temperature_2','room_temperature'],)

In [28]:
import logging

# Get the logger you used in your TimeSeries class
logger = logging.getLogger("forcateri.data.timeseries")
logger.setLevel(logging.INFO)  # or logging.DEBUG for more verbosity

# Create handler if it doesn't exist
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# Optional: prevent propagation to avoid double logs
logger.propagate = False


In [29]:
baltbest.get_data()


2025-07-03 09:41:24,059 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:24,059 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:24,064 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:24,065 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:24,069 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:24,070 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:24,074 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:24,075 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:24,079 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:24,080 - INFO - TimeSeries initialized from in

[TimeSeries(data=feature                    q_hca temperature_outdoor_avg temperature_1_max  \
 representation             value                   value             value   
 offset time_stamp                                                            
 0 days 2021-01-01 00:00:00   0.0                     NaN             16.78   
        2021-01-01 01:00:00   0.0                     NaN             16.75   
        2021-01-01 02:00:00   0.0                     NaN             16.74   
        2021-01-01 03:00:00   0.0                     NaN             16.72   
        2021-01-01 04:00:00   0.0                     NaN             16.71   
 ...                          ...                     ...               ...   
        2021-01-09 19:00:00   0.0                     NaN             16.40   
        2021-01-09 20:00:00   0.0                     NaN             16.42   
        2021-01-09 21:00:00   0.0                     NaN             16.43   
        2021-01-09 22:00:00   0.0   

In [30]:
roles = {
    'q_hca': SeriesRole.TARGET, 
    'temperature_outdoor_avg':SeriesRole.KNOWN, 
    'temperature_1_max':SeriesRole.OBSERVED, 
    'temperature_2_max':SeriesRole.OBSERVED,
    'temperature_room_avg':SeriesRole.OBSERVED,}
#['temperature_1_max', 'temperature_2_max','temperature_room_avg']
# roles = {
#     'delta': SeriesRole.TARGET, 
#     'outside_temp':SeriesRole.KNOWN, 
#     'max_temperature_1':SeriesRole.OBSERVED, 
#     'max_temperature_1':SeriesRole.OBSERVED,
#     'room_temperature':SeriesRole.OBSERVED,}

In [31]:
start = pd.Timestamp(2021, 1, 3, 4,tz=0)
end = pd.Timestamp(2021, 1, 8, 4,tz=0)
dataprovider = DataProvider(data_sources=[baltbest], roles=roles)

2025-07-03 09:41:26,366 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:26,367 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:26,370 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:26,370 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:26,374 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:26,374 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:26,378 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:26,378 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:26,383 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:26,383 - INFO - TimeSeries initialized from in

In [32]:
train = dataprovider.get_train_set()
val = dataprovider.get_val_set()
test = dataprovider.get_test_set()

2025-07-03 09:41:29,883 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:29,885 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:29,887 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:29,887 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:29,889 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:29,890 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:29,892 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:29,893 - INFO - TimeSeries initialized from internal-format DataFrame.
2025-07-03 09:41:29,895 - INFO - Checking DataFrame for deterministic representation. Feature levels must be unique.
2025-07-03 09:41:29,897 - INFO - TimeSeries initialized from in

In [33]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
quantiles = [0.1, 0.5, 0.9]
#dartstcn = DartsTCNModel( quantiles=quantiles)

In [34]:
model_path = '/home/dior00002/dfki/forcateri/_data/dartstcn/dartstcn.pt'
dartstcn = DartsTCNModel.load(model_path)
dartstcn.quantiles = quantiles
num_forecast_steps = 2
prediction = dartstcn.predict(
    data=test, # Pass the series used for prediction context
    predict_likelihood_parameters = True,
    n=num_forecast_steps,
)

/home/dior00002/dfki/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/dior00002/dfki/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/dior00002/dfki/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/dior00002/dfki/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/dior00002/dfki/forcateri/forcateri/mod

Predicting: |          | 0/? [00:00<?, ?it/s]

In [42]:
num_forecast_steps = 5
#num_monte_carlo_samples = 50 # A sufficient number of samples to approximate the quantiles

#Here prediction is darts timeseries
prediction = dartstcn.predict(
    data=val, # Pass the series used for prediction context
    predict_likelihood_parameters = True,
    n=num_forecast_steps,
)

/home/dior00002/dfki/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/dior00002/dfki/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/dior00002/dfki/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/dior00002/dfki/forcateri/forcateri/model/dartsmodels/dartsmodeladapter.py:99: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_reset = df_reset.drop(columns="offset")
/home/dior00002/dfki/forcateri/forcateri/mod

Predicting: |          | 0/? [00:00<?, ?it/s]

In [43]:
prediction[0]

<TimeSeries (time_stamp: 5, component: 3, sample: 1)> Size: 120B
array([[[nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan]]])
Coordinates:
  * time_stamp  (time_stamp) datetime64[ns] 40B 2021-01-07T01:00:00 ... 2021-...
  * component   (component) object 24B 'q_hca_q0.10' 'q_hca_q0.50' 'q_hca_q0.90'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
    metadata:           None

In [37]:
from IPython.display import display, Markdown
def mprint(s): display(Markdown(s))

In [44]:
prediction_ts = dartstcn.to_time_series(prediction)

2025-07-03 10:05:55,353 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03 10:05:55,356 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03 10:05:55,360 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03 10:05:55,363 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03 10:05:55,367 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03 10:05:55,370 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03 10:05:55,374 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03 10:05:55,378 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03 10:05:55,381 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03 10:05:55,385 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03 10:05:55,387 - INFO - TimeSeries initialized from compatible-format DataFrame.
2025-07-03

In [46]:
prediction_ts[35].data

feature                                target                    
representation                            0.1       0.5       0.9
offset          time_stamp                                       
0 days 01:00:00 2021-01-07 01:00:00 -0.002699 -0.001052  0.011139
0 days 02:00:00 2021-01-07 02:00:00 -0.002514 -0.001186  0.011218
0 days 03:00:00 2021-01-07 03:00:00 -0.001776 -0.000206  0.003874
0 days 04:00:00 2021-01-07 04:00:00 -0.001675  0.000210  0.002107
0 days 05:00:00 2021-01-07 05:00:00 -0.003138 -0.002049  0.067769

In [40]:
prediction_df = prediction[0].to_dataframe()
prediction_ts = TimeSeries(prediction_df,representation="quantile", quantiles=[0.1,0.5,0.9])
prediction_ts.data

2025-07-03 09:41:46,246 - INFO - TimeSeries initialized from compatible-format DataFrame.


feature                    target        
representation                0.1 0.5 0.9
offset time_stamp                        
0 days 2021-01-07 01:00:00    NaN NaN NaN
       2021-01-07 02:00:00    NaN NaN NaN